In [21]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


### About dataset

This is Kaggle dataset taken from Home Credit - Credit Risk Model Stability competition(https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability).

Overview
The goal of this competition is to predict which clients are more likely to default on their loans. The evaluation will favor solutions that are stable over time.

Your participation may offer consumer finance providers a more reliable and longer-lasting way to assess a potential client’s default risk.

In [12]:
# Открываем файл для чтения с использованием контекстного менеджера
with open("data_path.txt", "r") as file:
    # Читаем содержимое файла
    data_path = file.read()
data_path

'D:/data/kaggle_comp1'

In [10]:
file_name = "feature_definitions.csv"

file_path = os.path.join(data_path, file_name)

# Прочитайте CSV файл в DataFrame с помощью pandas
df = pd.read_csv(file_path)

df

,Variable,Description
0,actualdpd_943P,Days Past Due (DPD) of previous contract (actu...
1,actualdpdtolerance_344P,DPD of client with tolerance.
2,addres_district_368M,District of the person's address.
3,addres_role_871L,Role of person's address.
4,addres_zip_823M,Zip code of the address.
...,...,...
460,totinstallast1m_4525188A,Total amount of monthly instalments paid in th...
461,twobodfilling_608L,Type of application process.
462,type_25L,Contact type of a person.
463,typesuite_864L,Persons accompanying the client during the loa...


In [19]:
file_name = "csv_files/train/train_applprev_1_0.csv"

file_path = os.path.join(data_path, file_name)

# Прочитайте CSV файл в DataFrame с помощью pandas
data1 = pd.read_csv(file_path, dtype={27: str})

data1

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,2,0.0,640.2,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
1,2,0.0,1682.4,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,1,NaN,12.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,12.0
2,3,0.0,6140.0,NaN,NaN,P94_109_143,NaN,2019-01-07,NaN,0.0,...,0,NaN,12.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,12.0
3,4,0.0,2556.6,NaN,NaN,P24_27_36,NaN,2019-01-08,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,24.0
4,5,0.0,NaN,NaN,NaN,P85_114_140,NaN,2019-01-16,NaN,NaN,...,0,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3887679,2651092,0.0,0.0,NaN,NaN,P94_109_143,NaN,2018-05-04,NaN,100000.0,...,3,NaN,NaN,P46_145_78,a55475b1,P99_56_166,P94_109_143,NaN,D,NaN
3887680,2651092,0.0,4366.0,2017-11-09,NaN,a55475b1,NaN,2017-11-09,NaN,0.0,...,4,0.0,5.0,P60_146_156,a55475b1,a55475b1,a55475b1,NaN,K,5.0
3887681,2651092,0.0,2632.2,2015-12-11,1.0,a55475b1,1.0,2015-12-11,NaN,0.0,...,5,0.0,14.0,P46_145_78,a55475b1,a55475b1,a55475b1,NaN,K,14.0
3887682,2651092,0.0,NaN,2014-02-15,60000.0,a55475b1,1.0,2014-02-15,NaN,53300.0,...,6,0.0,NaN,P46_145_78,a55475b1,a55475b1,a55475b1,680531000.0,K,NaN


In [20]:
data1.describe()


,case_id,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
count,3.887684e+06,3.885450e+06,3.731833e+06,991660.000000,1.933791e+06,1.681780e+05,3.768614e+06,1.681780e+05,168178.000000,168178.000000,3.764355e+06,2.617307e+06,3.764355e+06,3.851072e+06,2.070306e+06,3.887684e+06,2.609762e+06,3.574851e+06,1.566510e+05,3.574851e+06
mean,1.397916e+06,1.056300e-02,3.413166e+03,19796.484030,8.434702e-01,2.026958e+04,3.254677e+03,-3.288887e+03,-6554.784203,0.522125,3.865786e+04,5.229101e+03,4.578899e+02,4.002969e+04,1.326582e+01,3.910262e+00,6.994588e+03,1.578210e+01,7.403548e+08,1.578210e+01
std,7.601594e+05,3.754277e+00,2.828269e+03,30687.652507,1.209425e+00,2.600278e+04,1.406135e+04,2.808611e+04,16888.862439,2.951672,3.754434e+04,1.927842e+04,2.697226e+03,3.139637e+04,1.348893e+02,4.101353e+00,2.916248e+04,1.046206e+01,5.498686e+07,1.046206e+01
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-1.140860e+05,0.000000e+00,-1.961082e+05,-206808.170000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,5.403424e+08,3.000000e+00
25%,1.251506e+06,0.000000e+00,1.710600e+03,1.000000,0.000000e+00,3.030500e+00,0.000000e+00,0.000000e+00,-1042.079025,0.000000,1.399800e+04,0.000000e+00,0.000000e+00,1.800000e+04,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,7.406884e+08,6.000000e+00
50%,1.451959e+06,0.000000e+00,2.761200e+03,5000.000000,0.000000e+00,1.258500e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,2.700000e+04,0.000000e+00,0.000000e+00,3.400000e+04,0.000000e+00,3.000000e+00,0.000000e+00,1.200000e+01,7.602487e+08,1.200000e+01
75%,1.641585e+06,0.000000e+00,4.396400e+03,30000.000000,1.000000e+00,3.044600e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,5.000000e+04,0.000000e+00,0.000000e+00,5.200000e+04,1.000000e+00,6.000000e+00,0.000000e+00,2.400000e+01,7.607250e+08,2.400000e+01
max,2.651092e+06,3.676000e+03,1.051302e+05,200000.000000,2.000000e+01,2.540730e+06,4.000000e+05,7.988198e+06,199567.000000,155.000000,7.153920e+05,5.074297e+05,3.204000e+05,1.960000e+05,4.058000e+03,1.900000e+01,1.210629e+06,6.200000e+01,7.808654e+08,6.200000e+01


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10,8))

for i, ax in enumerate(axes.ravel()):
    # ax.hist(data1.iloc[:, i], bins=15)
    ax.hist(data1.iloc[:, i].astype(str), bins=15)
    ax.set_xlabel(data1.columns[i])
    ax.set_ylabel('Frequency')
    # ax.axvline(data1.iloc[:, i].mean(), color='r', linestyle='dashed', linewidth=1)
    ax.axvline(data1.iloc[:, i].mean(skipna=True), color='r', linestyle='dashed', linewidth=1)

    ax.axvline(data1.iloc[:, i].median(), color='g', linestyle='dashed', linewidth=1)

plt.show()